<a href="https://colab.research.google.com/github/rahiakela/deep-learning-for-nlp-by-jason-brownlee/blob/part-1-data-preparation/3_preparing_text_data_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Text Data With Keras

You cannot feed raw text directly into deep learning models. Text data must be encoded as
numbers to be used as input or output for machine learning and deep learning models, such
as word embeddings. The Keras deep learning library provides some basic tools to help you
prepare your text data.

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Split Words with text to word sequence

A good first step when working with text is to split it into words. Words are called tokens and the process of splitting text into tokens is called tokenization. Keras provides the
text to word sequence() function that you can use to split text into a list of words. 
By default, this function automatically does 3 things:

* Splits words by space.
* Filters out punctuation.
* Converts text to lowercase (lower=True).

In [3]:
# define the document
text = 'The quick brown fox jumped over the lazy dog.'

# tokenize the document
result = text_to_word_sequence(text)
print(result)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']


## Encoding with one hot